# Library

In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from datetime import datetime

# fold path

In [1]:
ods_fold = 'D:/data/big2/ODS'
std_fold = 'D:/data/big2/STD'
fold = 'D:/data/big2'
an_fold = 'C:/Users/kbjung/Documents/GitHub/wabotech/low_gas_vehicle/analysis'

# Load

## 등록정보(2022.12.28)

In [5]:
# 11m 23s
# car = pd.read_csv(os.path.join(ods_fold, 'ODS_CEG_CAR_MIG.csv'), low_memory=False, usecols=['EMIS_CRTCNO', 'FST_REG_DE', 'VHCL_ERSR_YN', 'VIN', 'SRCMNNO', 'EMIS_GRD'])
car = pd.read_csv(os.path.join(std_fold, 'STD_CEG_CAR_MIG(20230821).csv'), low_memory=False, usecols=['EXHST_GAS_CERT_NO', 'FRST_REG_YMD', 'VHCL_ERSR_YN', 'VIN', 'MANP_MNG_NO', 'EXHST_GAS_GRD_CD', 'VHCTY_CD', 'PURPS_CD2', 'VHCL_OWNR_CL_CD'])
car.info()

ParserError: Error tokenizing data. C error: out of memory

In [ ]:
# cdict = {
#     'UPDT_DT':'수정일시',
#     'OWNR_SE':'소유자구분',
#     'VHRNO':'차량등록번호',
#     'BSPL_LEDO_CD':'본거지법정동코드',
#     'CRTN_DT':'생성일시',
#     'PURPS':'용도',
#     'EMIS_INSP_EFCT_DE':'배출가스검사유효일자',
#     'EMIS_CRTCNO':'배출가스인증번호',
#     'NOW_MBERNO':'현재회원번호',
#     'ACQS_DE':'취득일자',
#     'ACQS_AMT':'취득금액',
#     'FST_MBERNO':'최초회원번호',
#     'FST_REG_DE':'최초등록일자',
#     'VHCTY':'차종',
#     'VHRNO_NUM_INFO':'차량등록번호숫자정보',
#     'VHMNO':'차량관리번호',
#     'VHCL_MNFCT_DE':'차량제작일자',
#     'VHCL_YRIDNW':'차량연식',
#     'VHCL_ERSR_YN':'차량말소YN',
#     'VIN':'차대번호',
#     'SRCMNNO':'제원관리번호',
#     'EMIS_GRD':'배출가스등급',
#     'LEM_KND':'저공해조치종류',
#     'LEM_STRCHG_YN':'저공해조치구조변경YN',
#     'LEM_YN':'저공해조치YN(미사용)',
#     'NOW_OWNR_NM':'미정',
#     'BSPL_ADRS':'미정',
#     'OWNR_ADRS':'미정',
#     'BSPL_ZIP':'미정',
#     'FST_OWNR_NM':'미정',
#     'RMK':'미정',
#     'USER_ID':'미정',
# }
cdict = {
    'EXHST_GAS_CERT_NO':'배출가스인증번호',
    'FRST_REG_YMD':'최초등록일자',
    'VHCL_ERSR_YN':'차량말소YN',
    'VIN':'차대번호',
    'MANP_MNG_NO':'제원관리번호',
    'EXHST_GAS_GRD_CD':'배출가스등급',
    'VHCTY_CD':'차종', 
    'PURPS_CD2':'용도',
    'VHCL_OWNR_CL_CD':'사용자구분', 
}
len(cdict)

32

In [ ]:
carr = car.rename(columns=cdict)
carr.columns

Index(['차대번호', '최초등록일자', '제원관리번호', '배출가스인증번호', '배출가스등급', '차량말소YN'], dtype='object')

In [29]:
carr.shape

(29534047, 6)

In [30]:
# 중복 차대번호 제거
carr = carr.sort_values('최초등록일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
carr.shape

(29521505, 6)

In [31]:
carr['배출가스등급'].value_counts(dropna=False)

배출가스등급
2      11798397
3       9886814
1       2869999
5       2377881
4       1897535
X        303663
T2       250264
T3       136832
T5           47
T4           42
NaN          31
Name: count, dtype: int64

In [ ]:
## 배출가스등급 코드 변환
grd_dict = {
    'A0501':'1', 
    'A0502':'2', 
    'A0503':'3', 
    'A0504':'4', 
    'A0505':'5', 
    'A05T2':'2',
    'A05T3':'3', 
    'A05T4':'4', 
    'A05T5':'5', 
    'A05X':'X', 
}
carr['배출가스등급'] = carr['배출가스등급'].replace(grd_dict)
carr['배출가스등급'].unique()

In [32]:
carr['배출가스등급'].unique()

array(['X', '1', '2', '3', 'T2', 'T3', 'T4', '5', '4', 'T5', nan],
      dtype=object)

In [ ]:
## 차종 코드 변환
cd_dict = {
    'A31M':'이륜', 
    'A31P':'승용', 
    'A31S':'특수', 
    'A31T':'화물', 
    'A31V':'승합'
}
carr['차종'] = carr['차종'].replace(cd_dict)

In [ ]:
## 용도 코드 변환
purps_dict = {
    'A08P':'개인용', 
    'A08B':'영업용', 
    'A08O':'관용',
}
carr['용도'] = carr['용도'].replace(purps_dict)

In [ ]:
## 소유자구분 코드 변환
ownr_dict = {
    'A27B':'사업자', 
    'A27F':'외국인', 
    'A27L':'법인', 
    'A27O':'기타', 
    'A27R':'주민', 
}
carr['소유자구분'] = carr['소유자구분'].replace(ownr_dict)

## 제원정보

In [8]:
# 
# src = pd.read_csv(os.path.join(ods_fold, 'ODS_CEG_CAR_SRC_MIG.csv'), low_memory=False, usecols=['SRCMNNO', 'FUEL', 'VHCNM', 'VHCTY', 'VHCTY_TY'])
src = pd.read_csv(os.path.join(std_fold, 'STD_CEG_CAR_SRC_MIG(20230821).csv'), low_memory=False, usecols=['MANP_MNG_NO', 'FUEL_CD', 'VHCNM', 'VHCTY_CD', 'VHCTY_CL_CD', 'VHCTY_TY_CD2'])
src.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 566615 entries, 0 to 566614
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   FUEL_CD       545937 non-null  object
 1   MANP_MNG_NO   566615 non-null  object
 2   VHCNM         566544 non-null  object
 3   VHCTY_CD      566607 non-null  object
 4   VHCTY_CL_CD   551140 non-null  object
 5   VHCTY_TY_CD2  553038 non-null  object
dtypes: object(6)
memory usage: 25.9+ MB


In [9]:
# cdict = {
#     'UPDT_DT':'수정일시',
#     'CNFRM_DE':'승인일자',
#     'DELE_YN':'삭제YN', 
#     'CRTN_DT':'생성일시', 
#     'TOTL_WGHT':'총중량',
#     'VHCTY_TY':'차종유형(소분류)', 
#     'VHCTY_CL':'차종분류',
#     'VHCTY':'차종', 
#     'VHCNM':'차명',
#     'VHCL_HG':'차량높이',
#     'VHCL_WH':'차량너비',
#     'VHCL_LT':'차량길이',
#     'DRV_FRM':'구동형식',
#     'MKR_NM':'제작사명',
#     'SRCMNNO':'제원관리번호',
#     'CRYNG_WGHT':'적재중량',
#     'CAR_FRM':'자동차형식',
#     'EMPV_WGHT':'공차중량',
#     'PLOR_NM':'원산지명',
#     'FUEL':'연료',
#     'EGINTY':'엔진형식',
#     'EGINPWR':'엔진출력',
#     'TKCAR_NMPR':'승차인원',
#     'DSPLVL':'배기량',
#     'GEARBOX_KND':'변속기종류',
#     'REPCAR_NM':'미정',
#     'RMK':'미정',
#     'USER_ID':'미정',
# }
cdict = {
    'MANP_MNG_NO':'제원관리번호',
    'FUEL_CD':'연료',
    'VHCNM':'차명',
    'VHCTY_CD':'차종', 
    'VHCTY_CL_CD':'차종분류',
    'VHCTY_TY_CD2':'차종유형', 
}
srcr = src.rename(columns=cdict)
srcr.columns

Index(['연료', '제원관리번호', '차명', '차종', '차종분류', '차종유형'], dtype='object')

In [10]:
srcr.shape

(566615, 6)

In [11]:
srcr['차종'].unique()

array(['A31P', 'A31T', 'A31V', 'A31S', nan], dtype=object)

In [12]:
srcr['차종'].value_counts(dropna=False)

차종
A31P    245417
A31T    232843
A31V     64950
A31S     23397
NaN          8
Name: count, dtype: int64

In [13]:
## 차종 코드 변환
cd_dict = {
    'A31M':'이륜', 
    'A31P':'승용', 
    'A31S':'특수', 
    'A31T':'화물', 
    'A31V':'승합'
}
srcr['차종'] = srcr['차종'].replace(cd_dict)

In [14]:
srcr['차종'].value_counts(dropna=False)

차종
승용     245417
화물     232843
승합      64950
특수      23397
NaN         8
Name: count, dtype: int64

In [20]:
srcr['차종분류'].unique()

array(['A2826', 'A2814', 'A2825', 'A2819', 'A2821', 'A2824', 'A2811',
       'A2820', 'A2810', 'A2809', 'A2816', 'A2815', 'A2823', 'A2822',
       'A2813', 'A2812', nan], dtype=object)

In [23]:
## 차종유형 코드 변환
cl_dict = {
    'A2801':'견인', 
    'A2802':'구난', 
    'A2803':'기타', 
    'A2804':'다목적',
    'A2805':'덤프',
    'A2806':'밴',
    'A2807':'승용겸화물',
    'A2808':'일반',
    'A2809':'일반형-카고',
    'A2810':'일반형-픽업',
    'A2811':'특수',
    'A2812':'특수용도형',
    'A2813':'특수용도형-탱크로리',
    'A2814':'특수용도형-특수용도형',
    'A2815':'특수용도형-피견인',
    'A2816':'특수작업형',
    'A2818':'특수', 
    'A2819':'다목적', 
    'A2820':'덤프', 
    'A2821':'기타', 
    'A2822':'견인', 
    'A2823':'구난', 
    'A2824':'밴', 
    'A2825':'승용겸화물', 
    'A2826':'일반', 

}
srcr['차종분류'] = srcr['차종분류'].replace(cl_dict)

In [24]:
srcr['차종분류'].unique()

array(['일반', '특수용도형-특수용도형', '승용겸화물', '다목적', '기타', '밴', '특수', '덤프',
       '일반형-픽업', '일반형-카고', '특수작업형', '특수용도형-피견인', '구난', '견인', '특수용도형-탱크로리',
       '특수용도형', nan], dtype=object)

In [25]:
srcr['차종유형'].unique()

array(['대형', '중형', '소형', '경형', nan], dtype=object)

In [16]:
srcr['차종유형'].value_counts(dropna=False)

차종유형
A30L    356019
A30M    132073
A30S     55204
NaN      13577
A30C      9742
Name: count, dtype: int64

In [17]:
## 차종유형 코드 변환
ty_dict = {
    'A30C':'경형', 
    'A30L':'대형', 
    'A30M':'중형', 
    'A30S':'소형',
}
srcr['차종유형'] = srcr['차종유형'].replace(ty_dict)

In [18]:
srcr['차종유형'].value_counts(dropna=False)

차종유형
대형     356019
중형     132073
소형      55204
NaN     13577
경형       9742
Name: count, dtype: int64

In [19]:
srcr['연료'].unique()

array(['A90GS', 'A91DS', 'A94OT', 'A92LP', nan, 'A90GH', 'A92CN', 'A91KS',
       'A93SH', 'A91DH', 'A93EV', 'A90AC', 'A92LH', 'A93HD', 'A92LN',
       'A92CH'], dtype=object)

In [26]:
srcr['연료'].value_counts(dropna=False)

연료
A91DS    284477
A90GS    218330
A94OT     23793
NaN       20678
A92CN      7380
A92LP      4939
A90GH      4149
A93EV      2212
A91DH       361
A92CH       105
A93HD        98
A92LN        25
A91KS        20
A92LH        19
A90AC        17
A93SH        12
Name: count, dtype: int64

In [27]:
## 연료 코드 변환
fuel_dict = {
    'A90GS':'휘발유', 
    'A91DS':'경유',
    'A92LP':'LPG(액화석유가스)', 
    'A90GH':'휘발유 하이브리드', 
    'A93EV':'전기', 
    'A91DH':'경유 하이브리드', 
    'A92CN':'CNG(압축천연가스)', 
    'A93HD':'수소', 
    'A92LH':'LPG 하이브리드', 
    'A94OT':'기타연료', 
    'A92CH':'CNG 하이브리드',
    'A90AC':'알코올', 
    'A93SH':'태양열', 
    'A91KS':'등유', 
    'A92LN':'LNG(액화천연가스)', 
    'A90PH':'플러그인 하이브리드', 
}
srcr['연료'] = srcr['연료'].replace(fuel_dict)

In [28]:
srcr['연료'].value_counts(dropna=False)

연료
경유             284477
휘발유            218330
기타연료            23793
NaN             20678
CNG(압축천연가스)      7380
LPG(액화석유가스)      4939
휘발유 하이브리드        4149
전기               2212
경유 하이브리드          361
CNG 하이브리드         105
수소                 98
LNG(액화천연가스)        25
등유                 20
LPG 하이브리드          19
알코올                17
태양열                12
Name: count, dtype: int64

## 등록&제원 병합

In [56]:
carr.shape

(29521505, 6)

In [86]:
df = carr.merge(srcr, on='제원관리번호', how='left')
df.shape

(29521505, 10)

### [출력] 등록&제원(중복차대제거)(2022.12.28 기준)
- 중복 차대번호 제거

In [58]:
# 1m 50s
df.to_csv(os.path.join(ods_fold, '등록&제원(중복차대제거)(2022.12.28).csv'), index=False)

## 말소제거

In [93]:
df['차량말소YN'].value_counts(dropna=False)

차량말소YN
N    25692471
Y     3829034
Name: count, dtype: int64

In [110]:
dfn = df[df['차량말소YN'] =='N'].reset_index(drop=True)
dfn.shape

(25692471, 10)

## 등급별 연료별 차량 현황

In [111]:
dfn['배출가스등급'].unique()

array(['X', '1', '2', '3', 'T2', 'T3', 'T4', '5', '4', 'T5'], dtype=object)

In [112]:
dfn['배출가스등급'].value_counts(dropna=False)

배출가스등급
2     10959798
3      8622913
1      2771362
4      1511102
5      1154668
X       288790
T2      247477
T3      136296
T4          37
T5          28
Name: count, dtype: int64

In [113]:
dfn.columns

Index(['차대번호', '최초등록일자', '제원관리번호', '배출가스인증번호', '배출가스등급', '차량말소YN', '차명', '차종',
       '차종유형(소분류)', '연료'],
      dtype='object')

In [114]:
carname_fuel_dict = {
    'HYUNDAI PRESTO':'휘발유',
    'TOYOTA CAMRY하이브리드':'휘발유 하이브리드',
    '구급차(도요다)':'경유',
    '대우11톤카고':'경유',
    '대우8톤카고':'경유',
    '도요다웨곤엠브란스':'경유',
    '도요다코로나':'휘발유',
    '도요다크라운로얄싸롱':'휘발유',
    '도요타 프리우스(하이브리드)':'휘발유 하이브리드',
    '로얄살롱':'휘발유',
    '로얄살롱2.0AT':'휘발유',
    '르망':'휘발유',
    '르망펜타-5':'휘발유',
    '맥스디젤삼방':'경유',
    '머큐리 세이블':'경유',
    '벤즈 230E':'휘발유',
    '벤즈230E':'휘발유',
    '복사14':'휘발유',
    '브리샤2':'휘발유',
    '새한버스':'경유',
    '스텔라':'휘발유',
    '스텔라1.6CXL':'휘발유',
    '스텔라오토매틱':'휘발유',
    '아시아AM707중형버스':'경유',
    '에어스트림탐지견수송트레일러':'무동력',
    '엑셀':'휘발유',
    '인타담프':'경유',
    '일산 육륜':"경유",
    '캐피탈':'휘발유',
    '코티나마크IV':'휘발유',
    '포니2':'휘발유',
    '프레스토오토매틱':'휘발유',
    '현대리어엔진버스':'경유',
    '홍성캠핑트레일러':'무동력',
    'BENZ S500':'휘발유',
    'BMW':'휘발유',
    'BMW 3-SERIES':'휘발유',
    'BMW 318I':'휘발유',
    'BMW 325CI':'휘발유',
    'BMW 325I':'휘발유',
    'BMW 325i':'휘발유',
    'BMW 328I':'휘발유',
    'BMW 525I':'휘발유',
    'BMW 528I':'휘발유',
    'BMW 530I':'휘발유',
    'BMW 5SERIES':'휘발유',
    'BMW 750Li':'휘발유',
    'BMW X3':'휘발유',
    'BMW X5':'휘발유',
    'BMW Z3':'휘발유',
    'BMW530I':'휘발유',
    'BUICK CENTURY':'휘발유',
    'BUICK LESABRE':'휘발유',
    'BUICK TERRAZA':'휘발유',
    'CADILLAC CTS':'휘발유',
    'CHEVROLET IMPALA':'휘발유',
    'CHEVROLET PRIZM':'휘발유',
    'CHRYSLER':'휘발유',
    'CHRYSLER 300 LTD':'휘발유',
    'CHRYSLER 300C':'경유',
    'CHRYSLER HEMI AWD':'휘발유',
    'CHRYSLER PT CRUISER':'휘발유',
    'CHRYSLER TOWN & COUNTRY(7)':'휘발유',
    'CIELO':'휘발유',
    'CORSICA':'휘발유',
    'COUGAR':'휘발유',
    'DODGE CALIBER':'휘발유',
    'DODGE CARAVAN':'휘발유',
    'DODGE CHARGER':'휘발유',
    'DODGE DAKOTA':'휘발유',
    'DODGE DURANGO':'휘발유',
    'DODGE GRAND CARAVAN':'휘발유',
    'DODGE NEON SXT':'휘발유',
    'EF소나타':'휘발유',
    'ESCORT':'휘발유',
    'FIREBIRD':'휘발유',
    'FORD ESCAPE':'휘발유',
    'FORD EXPEDITION':'휘발유',
    'FORD EXPEDITION(8)':'휘발유',
    'FORD EXPLORER':'휘발유',
    'FORD FOCUS':'휘발유',
    'FORD FREESTYLE(7)':'휘발유',
    'FORD FUSION':'휘발유',
    'FORD MUSTANG':'휘발유',
    'FORD RANGER XLT':'경유',
    'FORD WINDSTAR':'휘발유',
    'G.M.C':'휘발유',
    'G20':'휘발유',
    'GMC':'휘발유',
    'GMC JIMMY':'휘발유',
    'GMC SIERRA':'휘발유',
    'HONDA ACCORD':'휘발유',
    'HONDA CR-V':'휘발유',
    'HONDA CR-V EX':'휘발유',
    'HONDA PILOT':'휘발유',
    'HYUNDAI SONATA':'휘발유',
    'HYUNDAI/SONATA':'휘발유',
    'INFINITI G37S':'휘발유',
    'JAGUAR X-TYPE':'휘발유',
    'JEEP GRAND CHEROKEE':'휘발유',
    'JEEP LIBERTY':'휘발유',
    'JEEP PATRIOT':"경유",
    'KIA OPTIMA':'휘발유',
    'KIA SEDONA':'휘발유',
    'LEXUS ES330':'휘발유',
    'LEXUS ES350':'휘발유',
    'LEXUS LS430':'휘발유',
    'LEXUS RS330':'휘발유',
    'LEXUS RX330':'휘발유',
    'LINCOLN MKX':'휘발유',
    'MAZDA MAZDA6':'휘발유',
    'MAZDA SENTA':'휘발유',
    'MERCEDES BENZ':'휘발유',
    'MERCEDES-BENZ':'휘발유',
    'MERCEDES-BENZ CLS 550':'경유',
    'MERCEDES-BENZ E-350':'휘발유',
    'MERCEDES-BENZ E-CLASS':'휘발유',
    'MERCEDES-BENZ R-CLASS 3500':'경유',
    'MERCEDES-BENZ S 500':'휘발유',
    'MERCEDES-BENZ S65':'경유',
    'MERCURY MILAN':'휘발유',
    'NEW 그랜저XG':'휘발유',
    'NISSAN MURANO':'휘발유',
    'NISSAN ROGUE 3.5':'휘발유',
    'NISSAN XTERRA':'휘발유',
    'OLDSMOBILE BRAVADA':'휘발유',
    'OLDSMOBILESILHOUET':'휘발유',
    'PLYMOUTH':'휘발유',
    'PONTIAC SUNBIRD':'휘발유',
    'PONTIAC VIBE':'휘발유',
    'PONTIAC VIBE BASE':'휘발유',
    'PORSCHE':'휘발유',
    'ROYALE':'휘발유',
    'ROYALSALON':'휘발유',
    'RX7':'휘발유',
    'SANTAFE':'휘발유',
    'SEPHIA':'휘발유',
    'SILHOUET':'휘발유',
    'SPORTAGE':'휘발유',
    'STAREX':'휘발유',
    'SUNDANCE':'휘발유',
    'TAURUS':'휘발유',
    'TOYOTA 4-RUNNER':'경유',
    'TOYOTA AVALON':'휘발유',
    'TOYOTA COROLLA':'휘발유',
    'TOYOTA ESTIIM':'휘발유',
    'TOYOTA MATRIX':'휘발유',
    'TOYOTA SIENNA':'휘발유',
    'TOYOTA SR-5':'휘발유',
    'VISION':'휘발유',
    'VOLKSWAGEN JETTA':'휘발유',
    'VOLVO 7G1':'휘발유',
    'VOLVO 960':'휘발유',
    'VOLVO S60':'휘발유',
    'VOLVO S80':'휘발유',
    'VOLVO T5':'휘발유',
    'VOLVO XC60':'휘발유',
    'VOLVO XC90':'휘발유',
    '그랜저2.0':'휘발유',
    '그랜저XG300L':'휘발유',
    '기아마스타':'경유',
    '뉴복사담프':'경유',
    '뉴비틀2.0':'휘발유',
    '닛산실비아2.0':'휘발유',
    '닛산씨마그로리아':'휘발유',
    '다임24KL에틸렌글리콜트레일러':'무동력',
    '다찌람밴':'휘발유',
    '대우11톤카고트럭':'경유',
    '대흥저상트레일러':'무동력',
    '도오다':'휘발유',
    '도요다MR2':'휘발유',
    '도요다세라':'휘발유',
    '도요다슈프라':'휘발유',
    '도요다크라운':'휘발유',
    '라비타(LAVITA)':'휘발유',
    '로얄살롱':'휘발유',
    '르망':'휘발유',
    '링컨MARK7':'경유',
    '벤츠 C300':'휘발유',
    '벤츠190E':'휘발유',
    '벤츠C280':'휘발유',
    '벤츠E230':'휘발유',
    '벤츠S320':'휘발유',
    '블루버드':'휘발유',
    '사파리':'휘발유',
    '새턴':'휘발유',
    '새한2.5톤카고':'경유',
    '셀리카-GT':'휘발유',
    '소나타':'휘발유',
    '소나타1800오토매틱':'휘발유',
    '수프라':'휘발유',
    '시보레':'휘발유',
    '시보레카베리아':'휘발유',
    '쏘나타':'휘발유',
    '아큐라':'휘발유',
    '에스페로':'휘발유',
    '엑셀운전교습용차량':'휘발유',
    '원흥식 트럭':'경유',
    '제설차':'경유',
    '지프':'경유',
    '체로키':'휘발유',
    '캠리':'휘발유',
    '코르시카':'휘발유',
    '코우거':'휘발유',
    '코티나디럭스':'휘발유',
    '크로마터보i.e':'휘발유',
    '티코(TICO)':'휘발유',
    '포니':'휘발유',
    '포니2-L.P.G':'LPG(액화석유가스)',
    '포니2-LPG':'LPG(액화석유가스)',
    '포르쉐':'휘발유',
    '프레스토오토메틱':'휘발유',
    '프리무스':'휘발유 하이브리드',
    '한원40A콤비콘테이너트레일러':'무동력',
    '한원40피트구즈넥트레일러':'무동력',
    '현대3톤트럭':'경유',
    '현대포터':'경유',
    '혼다 CR-V':'경유',
}

In [115]:
len(carname_fuel_dict)

216

In [116]:
dfn['연료'].value_counts(dropna=False)

연료
휘발유            12176278
경유              9890757
LPG(액화석유가스)     1904172
휘발유 하이브리드       1095728
전기               368686
NaN              146763
경유 하이브리드          35777
CNG(압축천연가스)       30461
수소                28227
LPG 하이브리드         14352
기타연료                933
CNG 하이브리드           304
LNG(액화천연가스)          27
알코올                   3
태양열                   2
등유                    1
Name: count, dtype: int64

#### 차명, 연료 매칭

In [117]:
# # about 5m 
# for k, v in tqdm(carname_fuel_dict.items()):
#     dfn.loc[(dfn['차명'] == k) & ((dfn['연료'].isnull() == True) | (dfn['연료'].isnull() == '기타연료')), '연료'] = v

In [118]:
dfn['연료'].value_counts(dropna=False)

연료
휘발유            12176278
경유              9890757
LPG(액화석유가스)     1904172
휘발유 하이브리드       1095728
전기               368686
NaN              146763
경유 하이브리드          35777
CNG(압축천연가스)       30461
수소                28227
LPG 하이브리드         14352
기타연료                933
CNG 하이브리드           304
LNG(액화천연가스)          27
알코올                   3
태양열                   2
등유                    1
Name: count, dtype: int64

In [119]:
dfn.loc[(dfn['배출가스등급'] == '2') | (dfn['배출가스등급'] == 'T2'), '배출가스등급'] = '2'
dfn.loc[(dfn['배출가스등급'] == '3') | (dfn['배출가스등급'] == 'T3'), '배출가스등급'] = '3'
dfn.loc[(dfn['배출가스등급'] == '4') | (dfn['배출가스등급'] == 'T4'), '배출가스등급'] = '4'
dfn.loc[(dfn['배출가스등급'] == '5') | (dfn['배출가스등급'] == 'T5'), '배출가스등급'] = '5'
dfn.loc[(dfn['배출가스등급'] == 'X') | (dfn['배출가스등급'].isnull() == True), '배출가스등급'] = 'X'

In [120]:
dfn['배출가스등급'].unique()

array(['X', '1', '2', '3', '4', '5'], dtype=object)

In [121]:
dfn['배출가스등급'].value_counts(dropna=False)

배출가스등급
2    11207275
3     8759209
1     2771362
4     1511139
5     1154696
X      288790
Name: count, dtype: int64

In [122]:
grp = dfn.groupby(['배출가스등급', '연료'], dropna=False)['차대번호'].count().unstack('배출가스등급')
grp

배출가스등급,1,2,3,4,5,X
연료,,,,,,
CNG 하이브리드,287.0,17.0,NaN,NaN,NaN,NaN
CNG(압축천연가스),NaN,24307.0,5998.0,1.0,5.0,150.0
LNG(액화천연가스),NaN,25.0,NaN,NaN,NaN,2.0
LPG 하이브리드,14349.0,1.0,NaN,NaN,NaN,2.0
LPG(액화석유가스),461743.0,1171738.0,236584.0,26089.0,436.0,7582.0
경유,NaN,NaN,7583997.0,1152631.0,1124162.0,29967.0
경유 하이브리드,NaN,NaN,34190.0,NaN,NaN,1587.0
기타연료,NaN,NaN,3.0,39.0,25.0,866.0
등유,NaN,NaN,NaN,NaN,NaN,1.0


In [123]:
# 
grp.to_excel(os.path.join(an_fold, '통계_연료_등급별_차량현황(연료미수정).xlsx'))

# [로드] 등록&제원 정보 2022.12.28 기준 

In [7]:
# about 8m 22s
df = pd.DataFrame()
for i in tqdm(range(1, 6)):
    file_name = f'[ODS][G{i}]등록정보&제원정보(말소제거)(2022.12.28).csv'
    temp = pd.read_csv(os.path.join(ods_fold, file_name), encoding='cp949', low_memory=False)
    temp = temp.sort_values(['최초등록일자', '배출가스검사유효일자'], ascending=[False, False]).drop_duplicates('차대번호').reset_index(drop=True)
    df = pd.concat([df, temp], ignore_index=True)

100%|██████████| 5/5 [08:17<00:00, 99.58s/it] 


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25404385 entries, 0 to 25404384
Data columns (total 49 columns):
 #   Column        Dtype  
---  ------        -----  
 0   차량관리번호        object 
 1   차량등록번호        object 
 2   본거지법정동코드      int64  
 3   소유자구분         object 
 4   차종_등록정보       object 
 5   용도            object 
 6   현재회원번호        object 
 7   차대번호          object 
 8   최초등록일자        int64  
 9   차량연식          int64  
 10  차량제작일자        float64
 11  배출가스검사유효일자    float64
 12  제원관리번호        object 
 13  배출가스인증번호      object 
 14  배출가스등급        object 
 15  저공해조치YN(미사용)  object 
 16  저공해조치종류       float64
 17  저공해조치구조변경YN   object 
 18  차량등록번호숫자정보    int64  
 19  차량말소YN        object 
 20  최초회원번호        object 
 21  취득일자          object 
 22  취득금액          float64
 23  생성일시_등록정보     object 
 24  수정일시_등록정보     object 
 25  차명            object 
 26  차종_제원정보       object 
 27  차종분류          object 
 28  차종유형(소분류)     object 
 29  자동차형식         object 
 30  제작사명          ob

In [9]:
df.shape

(25404385, 49)

# [로드] 본거지법정동코드

In [10]:
file_name = '법정동코드_2022.10.14.xlsx'
code = pd.read_excel(os.path.join(fold, file_name))
code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46340 entries, 0 to 46339
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   법정동코드     46340 non-null  int64  
 1   시도        46340 non-null  object 
 2   시구군       46317 non-null  object 
 3   읍면동       45847 non-null  object 
 4   리         33682 non-null  object 
 5   지역3       0 non-null      float64
 6   폐지여부      46340 non-null  object 
 7   대기관리권역    46340 non-null  object 
 8   대기관리권역YN  46334 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 3.2+ MB


# [merge] 법정동코드 기준

In [11]:
# '본거지법정도코드'열 데이터 확인
df['본거지법정동코드'].isnull().sum()

0

In [12]:
df.shape

(25404385, 49)

In [13]:
# about 40s
tot = df.merge(code, left_on='본거지법정동코드', right_on='법정동코드', how='left')
tot.shape

(25404385, 58)

In [14]:
len(tot['차대번호'].unique())

25404384

In [15]:
tot['차대번호'].isnull().sum()

0

# 전처리

## 말소제거
- 차량말소YN

In [16]:
tot['차량말소YN'].value_counts(dropna=False)

N    25404385
Name: 차량말소YN, dtype: int64

In [17]:
tot.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '현재회원번호',
       '차대번호', '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자', '제원관리번호', '배출가스인증번호',
       '배출가스등급', '저공해조치YN(미사용)', '저공해조치종류', '저공해조치구조변경YN', '차량등록번호숫자정보',
       '차량말소YN', '최초회원번호', '취득일자', '취득금액', '생성일시_등록정보', '수정일시_등록정보', '차명',
       '차종_제원정보', '차종분류', '차종유형(소분류)', '자동차형식', '제작사명', '연료', '엔진형식', '공차중량',
       '총중량', '적재중량', '엔진출력', '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명',
       '삭제YN', '승인일자', '구동형식', '변속기종류', '생성일시_제원정보', '수정일시_제원정보', '법정동코드',
       '시도', '시구군', '읍면동', '리', '지역3', '폐지여부', '대기관리권역', '대기관리권역YN'],
      dtype='object')

In [18]:
tot['차종_등록정보'].value_counts()

승용    20978861
화물     3589389
승합      718770
특수      117351
Name: 차종_등록정보, dtype: int64

In [19]:
tot['차종_제원정보'].value_counts()

승용    20952919
화물     3597795
승합      744096
특수      109575
Name: 차종_제원정보, dtype: int64

In [20]:
tot['차종분류'].value_counts()

일반             14491814
다목적             5670454
일반형-카고          2321758
기타              1411153
특수용도형-특수용도형      487937
일반형-픽업           366795
밴                357182
승용겸화물            102410
덤프                53290
특수작업형             41819
견인                40565
특수                21186
특수용도형             15932
구난                11255
특수용도형-탱크로리        10832
특수용도형-피견인             3
Name: 차종분류, dtype: int64

In [21]:
tot['구동형식'].value_counts()

3.0     4529093
1.0     1247383
11.0     891913
4.0      447899
13.0     187605
2.0      114561
14.0      26427
12.0       7860
19.0       6161
18.0       6103
15.0       5265
30.0        227
31.0         50
16.0         24
17.0         10
20.0          5
Name: 구동형식, dtype: int64

In [22]:
tot['원산지명'].value_counts()

대한민국        22243095
독일           1500532
미국            558103
일본            300730
영국            161847
              ...   
칠레                 1
인도네시아              1
생피에르 미클롱           1
파키스탄               1
아이티                1
Name: 원산지명, Length: 65, dtype: int64

## 폐지여부 확인

In [23]:
tot['폐지여부'].value_counts(dropna=False)

존재     25322420
폐지        81963
NaN           2
Name: 폐지여부, dtype: int64

## 법정동코드 확인

In [24]:
tot['대기관리권역YN'].isnull().sum()

3

In [25]:
tot.loc[tot['대기관리권역YN'].isnull() == True, ['본거지법정동코드', '시도', '시구군', '읍면동', '리', '지역3', '차량말소YN', '차량연식', '배출가스등급', '차대번호', '폐지여부', '대기관리권역', '대기관리권역YN']]

,본거지법정동코드,시도,시구군,읍면동,리,지역3,차량말소YN,차량연식,배출가스등급,차대번호,폐지여부,대기관리권역,대기관리권역YN
24137513,4163055000,NaN,NaN,NaN,NaN,NaN,N,1992,4,KMHVE21NPNU122861,NaN,NaN,NaN
24922881,4165052000,NaN,NaN,NaN,NaN,NaN,N,2002,5,KMFGA17AP2C167888,NaN,NaN,NaN
25398594,4100000000,경기,NaN,NaN,NaN,NaN,N,1974,5,10742255732,존재,수도권,NaN


In [26]:
tot.loc[tot['본거지법정동코드'] == 4163055000, ['시도', '시구군', '대기관리권역', '대기관리권역YN',]] = ['경기', '양주시', '수도권', 'Y']
tot.loc[tot['본거지법정동코드'] == 4165052000, ['시도', '시구군', '대기관리권역', '대기관리권역YN',]] = ['경기', '포천시', '수도권', 'Y']
tot.loc[tot['본거지법정동코드'] == 4100000000, '대기관리권역YN'] = 'Y'

In [27]:
tot.loc[tot['대기관리권역YN'].isnull() == True, ['본거지법정동코드', '시도', '시구군', '읍면동', '리', '지역3', '차량말소YN', '차량연식', '배출가스등급', '차대번호', '폐지여부', '대기관리권역', '대기관리권역YN']]

,본거지법정동코드,시도,시구군,읍면동,리,지역3,차량말소YN,차량연식,배출가스등급,차대번호,폐지여부,대기관리권역,대기관리권역YN


In [28]:
tot['대기관리권역YN'].isnull().sum()

0

## 연료 확인

In [29]:
tot['연료'].isnull().sum()

264

In [30]:
tot['연료'].value_counts(dropna=False)

휘발유            12101061
경유              9860935
LPG(액화석유가스)     1896594
휘발유 하이브리드       1071375
전기               366804
경유 하이브리드          34191
CNG(압축천연가스)       30311
수소                28102
LPG 하이브리드         14350
CNG 하이브리드           304
NaN                 264
기타연료                 67
LNG(액화천연가스)          25
알코올                   2
Name: 연료, dtype: int64

### 기타연료의 차명으로 매칭

In [31]:
set(tot.loc[tot['연료'] == '기타연료', '차명'].unique())

{'HYUNDAI PRESTO',
 'TOYOTA CAMRY하이브리드',
 '구급차(도요다)',
 '대우11톤카고',
 '대우8톤카고',
 '도요다웨곤엠브란스',
 '도요다코로나',
 '도요다크라운로얄싸롱',
 '도요타 프리우스(하이브리드)',
 '로얄살롱',
 '로얄살롱2.0AT',
 '르망',
 '르망펜타-5',
 '맥스디젤삼방',
 '머큐리 세이블',
 '벤즈 230E',
 '벤즈230E',
 '복사14',
 '브리샤2',
 '새한버스',
 '스텔라',
 '스텔라1.6CXL',
 '스텔라오토매틱',
 '아시아AM707중형버스',
 '에어스트림탐지견수송트레일러',
 '엑셀',
 '인타담프',
 '일산 육륜',
 '캐피탈',
 '코티나마크IV',
 '포니2',
 '프레스토오토매틱',
 '현대리어엔진버스',
 '홍성캠핑트레일러'}

### 연료 결측값의 차명으로 매칭

In [32]:
set(tot.loc[tot['연료'].isnull() == True, '차명'].unique())

{'BENZ S500',
 'BMW',
 'BMW 3-SERIES',
 'BMW 318I',
 'BMW 325CI',
 'BMW 325I',
 'BMW 325i',
 'BMW 328I',
 'BMW 525I',
 'BMW 528I',
 'BMW 530I',
 'BMW 5SERIES',
 'BMW 750Li',
 'BMW X3',
 'BMW X5',
 'BMW Z3',
 'BMW530I',
 'BUICK CENTURY',
 'BUICK LESABRE',
 'BUICK TERRAZA',
 'CADILLAC CTS',
 'CHEVROLET IMPALA',
 'CHEVROLET PRIZM',
 'CHRYSLER',
 'CHRYSLER 300 LTD',
 'CHRYSLER 300C',
 'CHRYSLER HEMI AWD',
 'CHRYSLER PT CRUISER',
 'CHRYSLER TOWN & COUNTRY(7)',
 'CIELO',
 'CORSICA',
 'COUGAR',
 'DODGE CALIBER',
 'DODGE CARAVAN',
 'DODGE CHARGER',
 'DODGE DAKOTA',
 'DODGE DURANGO',
 'DODGE GRAND CARAVAN',
 'DODGE NEON SXT',
 'EF소나타',
 'ESCORT',
 'FIREBIRD',
 'FORD ESCAPE',
 'FORD EXPEDITION',
 'FORD EXPEDITION(8)',
 'FORD EXPLORER',
 'FORD FOCUS',
 'FORD FREESTYLE(7)',
 'FORD FUSION',
 'FORD MUSTANG',
 'FORD RANGER XLT',
 'FORD WINDSTAR',
 'G.M.C',
 'G20',
 'GMC',
 'GMC JIMMY',
 'GMC SIERRA',
 'HONDA ACCORD',
 'HONDA CR-V',
 'HONDA CR-V EX',
 'HONDA PILOT',
 'HYUNDAI SONATA',
 'HYUNDAI/SONATA

In [4]:
carname_fuel_dict = {'HYUNDAI PRESTO':'휘발유',
'TOYOTA CAMRY하이브리드':'휘발유 하이브리드',
'구급차(도요다)':'경유',
'대우11톤카고':'경유',
'대우8톤카고':'경유',
'도요다웨곤엠브란스':'경유',
'도요다코로나':'휘발유',
'도요다크라운로얄싸롱':'휘발유',
'도요타 프리우스(하이브리드)':'휘발유 하이브리드',
'로얄살롱':'휘발유',
'로얄살롱2.0AT':'휘발유',
'르망':'휘발유',
'르망펜타-5':'휘발유',
'맥스디젤삼방':'경유',
'머큐리 세이블':'경유',
'벤즈 230E':'휘발유',
'벤즈230E':'휘발유',
'복사14':'휘발유',
'브리샤2':'휘발유',
'새한버스':'경유',
'스텔라':'휘발유',
'스텔라1.6CXL':'휘발유',
'스텔라오토매틱':'휘발유',
'아시아AM707중형버스':'경유',
'에어스트림탐지견수송트레일러':'무동력',
'엑셀':'휘발유',
'인타담프':'경유',
'일산 육륜':"경유",
'캐피탈':'휘발유',
'코티나마크IV':'휘발유',
'포니2':'휘발유',
'프레스토오토매틱':'휘발유',
'현대리어엔진버스':'경유',
'홍성캠핑트레일러':'무동력',
'BENZ S500':'휘발유',
'BMW':'휘발유',
'BMW 3-SERIES':'휘발유',
'BMW 318I':'휘발유',
'BMW 325CI':'휘발유',
'BMW 325I':'휘발유',
'BMW 325i':'휘발유',
'BMW 328I':'휘발유',
'BMW 525I':'휘발유',
'BMW 528I':'휘발유',
'BMW 530I':'휘발유',
'BMW 5SERIES':'휘발유',
'BMW 750Li':'휘발유',
'BMW X3':'휘발유',
'BMW X5':'휘발유',
'BMW Z3':'휘발유',
'BMW530I':'휘발유',
'BUICK CENTURY':'휘발유',
'BUICK LESABRE':'휘발유',
'BUICK TERRAZA':'휘발유',
'CADILLAC CTS':'휘발유',
'CHEVROLET IMPALA':'휘발유',
'CHEVROLET PRIZM':'휘발유',
'CHRYSLER':'휘발유',
'CHRYSLER 300 LTD':'휘발유',
'CHRYSLER 300C':'경유',
'CHRYSLER HEMI AWD':'휘발유',
'CHRYSLER PT CRUISER':'휘발유',
'CHRYSLER TOWN & COUNTRY(7)':'휘발유',
'CIELO':'휘발유',
'CORSICA':'휘발유',
'COUGAR':'휘발유',
'DODGE CALIBER':'휘발유',
'DODGE CARAVAN':'휘발유',
'DODGE CHARGER':'휘발유',
'DODGE DAKOTA':'휘발유',
'DODGE DURANGO':'휘발유',
'DODGE GRAND CARAVAN':'휘발유',
'DODGE NEON SXT':'휘발유',
'EF소나타':'휘발유',
'ESCORT':'휘발유',
'FIREBIRD':'휘발유',
'FORD ESCAPE':'휘발유',
'FORD EXPEDITION':'휘발유',
'FORD EXPEDITION(8)':'휘발유',
'FORD EXPLORER':'휘발유',
'FORD FOCUS':'휘발유',
'FORD FREESTYLE(7)':'휘발유',
'FORD FUSION':'휘발유',
'FORD MUSTANG':'휘발유',
'FORD RANGER XLT':'경유',
'FORD WINDSTAR':'휘발유',
'G.M.C':'휘발유',
'G20':'휘발유',
'GMC':'휘발유',
'GMC JIMMY':'휘발유',
'GMC SIERRA':'휘발유',
'HONDA ACCORD':'휘발유',
'HONDA CR-V':'휘발유',
'HONDA CR-V EX':'휘발유',
'HONDA PILOT':'휘발유',
'HYUNDAI SONATA':'휘발유',
'HYUNDAI/SONATA':'휘발유',
'INFINITI G37S':'휘발유',
'JAGUAR X-TYPE':'휘발유',
'JEEP GRAND CHEROKEE':'휘발유',
'JEEP LIBERTY':'휘발유',
'JEEP PATRIOT':"경유",
'KIA OPTIMA':'휘발유',
'KIA SEDONA':'휘발유',
'LEXUS ES330':'휘발유',
'LEXUS ES350':'휘발유',
'LEXUS LS430':'휘발유',
'LEXUS RS330':'휘발유',
'LEXUS RX330':'휘발유',
'LINCOLN MKX':'휘발유',
'MAZDA MAZDA6':'휘발유',
'MAZDA SENTA':'휘발유',
'MERCEDES BENZ':'휘발유',
'MERCEDES-BENZ':'휘발유',
'MERCEDES-BENZ CLS 550':'경유',
'MERCEDES-BENZ E-350':'휘발유',
'MERCEDES-BENZ E-CLASS':'휘발유',
'MERCEDES-BENZ R-CLASS 3500':'경유',
'MERCEDES-BENZ S 500':'휘발유',
'MERCEDES-BENZ S65':'경유',
'MERCURY MILAN':'휘발유',
'NEW 그랜저XG':'휘발유',
'NISSAN MURANO':'휘발유',
'NISSAN ROGUE 3.5':'휘발유',
'NISSAN XTERRA':'휘발유',
'OLDSMOBILE BRAVADA':'휘발유',
'OLDSMOBILESILHOUET':'휘발유',
'PLYMOUTH':'휘발유',
'PONTIAC SUNBIRD':'휘발유',
'PONTIAC VIBE':'휘발유',
'PONTIAC VIBE BASE':'휘발유',
'PORSCHE':'휘발유',
'ROYALE':'휘발유',
'ROYALSALON':'휘발유',
'RX7':'휘발유',
'SANTAFE':'휘발유',
'SEPHIA':'휘발유',
'SILHOUET':'휘발유',
'SPORTAGE':'휘발유',
'STAREX':'휘발유',
'SUNDANCE':'휘발유',
'TAURUS':'휘발유',
'TOYOTA 4-RUNNER':'경유',
'TOYOTA AVALON':'휘발유',
'TOYOTA COROLLA':'휘발유',
'TOYOTA ESTIIM':'휘발유',
'TOYOTA MATRIX':'휘발유',
'TOYOTA SIENNA':'휘발유',
'TOYOTA SR-5':'휘발유',
'VISION':'휘발유',
'VOLKSWAGEN JETTA':'휘발유',
'VOLVO 7G1':'휘발유',
'VOLVO 960':'휘발유',
'VOLVO S60':'휘발유',
'VOLVO S80':'휘발유',
'VOLVO T5':'휘발유',
'VOLVO XC60':'휘발유',
'VOLVO XC90':'휘발유',
'그랜저2.0':'휘발유',
'그랜저XG300L':'휘발유',
'기아마스타':'경유',
'뉴복사담프':'경유',
'뉴비틀2.0':'휘발유',
'닛산실비아2.0':'휘발유',
'닛산씨마그로리아':'휘발유',
'다임24KL에틸렌글리콜트레일러':'무동력',
'다찌람밴':'휘발유',
'대우11톤카고트럭':'경유',
'대흥저상트레일러':'무동력',
'도오다':'휘발유',
'도요다MR2':'휘발유',
'도요다세라':'휘발유',
'도요다슈프라':'휘발유',
'도요다크라운':'휘발유',
'라비타(LAVITA)':'휘발유',
'로얄살롱':'휘발유',
'르망':'휘발유',
'링컨MARK7':'경유',
'벤츠 C300':'휘발유',
'벤츠190E':'휘발유',
'벤츠C280':'휘발유',
'벤츠E230':'휘발유',
'벤츠S320':'휘발유',
'블루버드':'휘발유',
'사파리':'휘발유',
'새턴':'휘발유',
'새한2.5톤카고':'경유',
'셀리카-GT':'휘발유',
'소나타':'휘발유',
'소나타1800오토매틱':'휘발유',
'수프라':'휘발유',
'시보레':'휘발유',
'시보레카베리아':'휘발유',
'쏘나타':'휘발유',
'아큐라':'휘발유',
'에스페로':'휘발유',
'엑셀운전교습용차량':'휘발유',
'원흥식 트럭':'경유',
'제설차':'경유',
'지프':'경유',
'체로키':'휘발유',
'캠리':'휘발유',
'코르시카':'휘발유',
'코우거':'휘발유',
'코티나디럭스':'휘발유',
'크로마터보i.e':'휘발유',
'티코(TICO)':'휘발유',
'포니':'휘발유',
'포니2-L.P.G':'LPG(액화석유가스)',
'포니2-LPG':'LPG(액화석유가스)',
'포르쉐':'휘발유',
'프레스토오토메틱':'휘발유',
'프리무스':'휘발유 하이브리드',
'한원40A콤비콘테이너트레일러':'무동력',
'한원40피트구즈넥트레일러':'무동력',
'현대3톤트럭':'경유',
'현대포터':'경유',
'혼다 CR-V':'경유',}

In [5]:
len(carname_fuel_dict)

216

#### [출력] 차명, 연료 매칭

In [6]:
carname_fuel_df = pd.DataFrame(carname_fuel_dict, index=['연료']).T.reset_index()
carname_fuel_df = carname_fuel_df.rename(columns={'index':'차명'})
carname_fuel_df = carname_fuel_df.sort_values('차명')
carname_fuel_df.head()

,차명,연료
34,BENZ S500,휘발유
35,BMW,휘발유
36,BMW 3-SERIES,휘발유
37,BMW 318I,휘발유
38,BMW 325CI,휘발유


In [8]:
today_date = datetime.today().strftime("%Y.%m.%d_%p.%H.%M.%S")
today_date

'2023.03.03_AM.11.12.03'

In [10]:
carname_fuel_df.to_excel(f'D:/data/low_gas_vehicle/차명_연료_매칭_{today_date}.xlsx', index=False)

In [34]:
# about 5m 
for k, v in tqdm(carname_fuel_dict.items()):
    # tot.loc[((tot['연료'].isnull() == True) | (tot['연료'] == '기타연료')) & (tot['차명'] == k), '연료'] = v
    tot.loc[tot['차명'] == k, '연료'] = v

100%|██████████| 216/216 [05:00<00:00,  1.39s/it]


In [36]:
tot['연료'].value_counts(dropna=False)

휘발유            12101949
경유              9860384
LPG(액화석유가스)     1896593
휘발유 하이브리드       1071363
전기               366804
경유 하이브리드          34191
CNG(압축천연가스)       30311
수소                28102
LPG 하이브리드         14350
CNG 하이브리드           304
LNG(액화천연가스)          25
무동력                   7
알코올                   2
Name: 연료, dtype: int64

### 최종 확인

In [37]:
tot.loc[tot['연료'].isnull() == True, ['차명', '연료', '엔진형식', '자동차형식']]

,차명,연료,엔진형식,자동차형식


## 임시등급 병합

In [38]:
tot['배출가스등급'].value_counts(dropna=False)

2     10960196
3      8623093
1      2771380
4      1511174
5      1154695
T2      247481
T3      136300
T4          37
T5          29
Name: 배출가스등급, dtype: int64

In [39]:
tot.shape

(25404385, 58)

In [40]:
grade_list = []
for one in tqdm(tot['배출가스등급']):
    if str(one) == '1':
        grade_list.append('1')
    elif str(one) == '2' or str(one) == 'T2':
        grade_list.append('2')
    elif str(one) == '3' or str(one) == 'T3':
        grade_list.append('3')
    elif str(one) == '4' or str(one) == 'T4':
        grade_list.append('4')
    elif str(one) == '5' or str(one) == 'T5':
        grade_list.append('5')
    elif str(one) == 'X':
        grade_list.append('X')
    else:
        grade_list.append(np.nan)
len(grade_list)

100%|██████████| 25404385/25404385 [00:27<00:00, 939557.63it/s] 


25404385

In [41]:
grade_list[:5]

['1', '1', '1', '1', '1']

In [42]:
tot['등급'] = grade_list

## 연료 결측값 샘플

In [43]:
# about 28s
total = tot.copy()

In [44]:
total[total['연료'].isnull() == True].shape

(0, 59)

In [45]:
total.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '현재회원번호',
       '차대번호', '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자', '제원관리번호', '배출가스인증번호',
       '배출가스등급', '저공해조치YN(미사용)', '저공해조치종류', '저공해조치구조변경YN', '차량등록번호숫자정보',
       '차량말소YN', '최초회원번호', '취득일자', '취득금액', '생성일시_등록정보', '수정일시_등록정보', '차명',
       '차종_제원정보', '차종분류', '차종유형(소분류)', '자동차형식', '제작사명', '연료', '엔진형식', '공차중량',
       '총중량', '적재중량', '엔진출력', '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명',
       '삭제YN', '승인일자', '구동형식', '변속기종류', '생성일시_제원정보', '수정일시_제원정보', '법정동코드',
       '시도', '시구군', '읍면동', '리', '지역3', '폐지여부', '대기관리권역', '대기관리권역YN', '등급'],
      dtype='object')

# [출력] 등록&제원정보(연료수정)(등급컬럼추가)

In [49]:
# about 11m 23s
total.to_csv(os.path.join(ods_fold, '[ODS]등록정보&제원정보(연료수정)(등급컬럼추가)(2022.12.28).csv'), index=False)

In [50]:
# about 6s
g4 = total[total['등급'] == '4'].reset_index(drop=True)
g5 = total[total['등급'] == '5'].reset_index(drop=True)
g4.shape, g5.shape

((1511211, 59), (1154724, 59))

In [51]:
# about 1m 9s
g4.to_csv(os.path.join(ods_fold, '[ODS][G4]등록정보&제원정보(연료수정)(등급컬럼추가)(2022.12.28).csv'), index=False)
g5.to_csv(os.path.join(ods_fold, '[ODS][G5]등록정보&제원정보(연료수정)(등급컬럼추가)(2022.12.28).csv'), index=False)

# [로드] 등록&제원정보(연료수정)(등급컬럼추가)

In [24]:
# about 7m 42s
total = pd.read_csv(os.path.join(ods_fold, '[ODS]등록정보&제원정보(연료수정)(등급컬럼추가)(2022.12.28).csv'), low_memory=False)

In [4]:
g5 = pd.read_csv(os.path.join(ods_fold, '[ODS][G5]등록정보&제원정보(연료수정)(등급컬럼추가)(2022.12.28).csv'), low_memory=False)

# 분석

##  연료별 통계

In [52]:
total.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '현재회원번호',
       '차대번호', '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자', '제원관리번호', '배출가스인증번호',
       '배출가스등급', '저공해조치YN(미사용)', '저공해조치종류', '저공해조치구조변경YN', '차량등록번호숫자정보',
       '차량말소YN', '최초회원번호', '취득일자', '취득금액', '생성일시_등록정보', '수정일시_등록정보', '차명',
       '차종_제원정보', '차종분류', '차종유형(소분류)', '자동차형식', '제작사명', '연료', '엔진형식', '공차중량',
       '총중량', '적재중량', '엔진출력', '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명',
       '삭제YN', '승인일자', '구동형식', '변속기종류', '생성일시_제원정보', '수정일시_제원정보', '법정동코드',
       '시도', '시구군', '읍면동', '리', '지역3', '폐지여부', '대기관리권역', '대기관리권역YN', '등급'],
      dtype='object')

In [53]:
total.groupby(['연료', '등급'], dropna=False)['차대번호'].count()

연료           등급
CNG 하이브리드    1         287
             2          17
CNG(압축천연가스)  2       24307
             3        5998
             4           1
             5           5
LNG(액화천연가스)  2          25
LPG 하이브리드    1       14349
             2           1
LPG(액화석유가스)  1      461745
             2     1171739
             3      236584
             4       26087
             5         438
경유           2          86
             3     7584030
             4     1153141
             5     1123127
경유 하이브리드     3       34191
무동력          3           1
             4           1
             5           5
수소           1       28102
알코올          4           1
             5           1
전기           1      366804
휘발유          1      969107
             2     9871132
             3      898587
             4      331976
             5       31147
휘발유 하이브리드    1      930986
             2      140370
             3           2
             4           4
             5           1
Name: 차대번호, 

In [54]:
total['등급'].isnull().sum()

0

In [55]:
total.groupby(['연료', '등급'], dropna=False)['차대번호'].count().unstack()

등급,1,2,3,4,5
연료,,,,,
CNG 하이브리드,287.0,17.0,NaN,NaN,NaN
CNG(압축천연가스),NaN,24307.0,5998.0,1.0,5.0
LNG(액화천연가스),NaN,25.0,NaN,NaN,NaN
LPG 하이브리드,14349.0,1.0,NaN,NaN,NaN
LPG(액화석유가스),461745.0,1171739.0,236584.0,26087.0,438.0
경유,NaN,86.0,7584030.0,1153141.0,1123127.0
경유 하이브리드,NaN,NaN,34191.0,NaN,NaN
무동력,NaN,NaN,1.0,1.0,5.0
수소,28102.0,NaN,NaN,NaN,NaN


## 5등급 경유 대기관리권역별 통계

In [56]:
g5 = total[total['등급'] == '5'].reset_index(drop=True)
g5.shape

(1154724, 59)

### 5등급 경유차만 추출

In [57]:
g5_diesel = g5[g5['연료'] == '경유'].reset_index(drop=True)
g5_diesel.shape

(1123127, 59)

In [58]:
g5_diesel.groupby(['대기관리권역', '시도'], dropna=False)['차대번호'].count()

대기관리권역  시도
그 외     강원     47853
        제주     24535
남부권     광주     25632
        전남     76539
동남권     경남    103660
        경북    120976
        대구     48974
        부산     65635
        울산     25873
수도권     경기    222192
        서울    116659
        인천     53366
중부권     대전     25732
        세종      4267
        전북     57496
        충남     61883
        충북     41855
Name: 차대번호, dtype: int64

In [59]:
g5_diesel['시도'].isnull().sum()

0

In [60]:
g5_diesel['대기관리권역'].value_counts(dropna=False)

수도권    392217
동남권    365118
중부권    191233
남부권    102171
그 외     72388
Name: 대기관리권역, dtype: int64

In [61]:
g5_diesel['대기관리권역YN'].value_counts(dropna=False)

Y    843131
N    279996
Name: 대기관리권역YN, dtype: int64

In [62]:
g5_diesel.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '현재회원번호',
       '차대번호', '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자', '제원관리번호', '배출가스인증번호',
       '배출가스등급', '저공해조치YN(미사용)', '저공해조치종류', '저공해조치구조변경YN', '차량등록번호숫자정보',
       '차량말소YN', '최초회원번호', '취득일자', '취득금액', '생성일시_등록정보', '수정일시_등록정보', '차명',
       '차종_제원정보', '차종분류', '차종유형(소분류)', '자동차형식', '제작사명', '연료', '엔진형식', '공차중량',
       '총중량', '적재중량', '엔진출력', '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명',
       '삭제YN', '승인일자', '구동형식', '변속기종류', '생성일시_제원정보', '수정일시_제원정보', '법정동코드',
       '시도', '시구군', '읍면동', '리', '지역3', '폐지여부', '대기관리권역', '대기관리권역YN', '등급'],
      dtype='object')

In [63]:
g5_y = g5_diesel[g5_diesel['대기관리권역YN'] == 'Y']
g5_y.shape

(843131, 59)

In [64]:
g5_n = g5_diesel[g5_diesel['대기관리권역YN'] == 'N']
g5_n.shape

(279996, 59)

In [65]:
g5_y.groupby(['대기관리권역', '시도'], dropna=False)['차대번호'].count()

대기관리권역  시도
남부권     광주     25632
        전남     32564
동남권     경남     60713
        경북     55035
        대구     48974
        부산     65635
        울산     25873
수도권     경기    213117
        서울    116659
        인천     52176
중부권     대전     25732
        세종      4267
        전북     26159
        충남     59082
        충북     31513
Name: 차대번호, dtype: int64

In [66]:
g5_n.groupby(['대기관리권역', '시도'], dropna=False)['차대번호'].count()

대기관리권역  시도
그 외     강원    47853
        제주    24535
남부권     전남    43975
동남권     경남    42947
        경북    65941
수도권     경기     9075
        인천     1190
중부권     전북    31337
        충남     2801
        충북    10342
Name: 차대번호, dtype: int64

### [출력] 연료별 통계, 5등급 대기관리권역별 통계

In [67]:
today_date = datetime.today().strftime("%Y.%m.%p_%H_%M_%S")
today_date

'2023.02.PM_14_54_33'

In [68]:
with pd.ExcelWriter(f'analysis/통계_연료별_5등급대기관리권역별{today_date}.xlsx') as writer:
    total.groupby(['연료', '등급'], dropna=False)['차대번호'].count().unstack().to_excel(writer, sheet_name='연료별')
    g5_y.groupby(['대기관리권역', '시도'], dropna=False)['차대번호'].count().to_excel(writer, sheet_name='대기관리권역별_y')
    g5_n.groupby(['대기관리권역', '시도'], dropna=False)['차대번호'].count().to_excel(writer, sheet_name='대기관리권역별_n')

## 4, 5등급 검사 기록없는 차량 수

## [로드] 정기&정밀검사

In [244]:
# about 4m 21s
file_name = '[ODS]정기&정밀검사(최신검사만)(2022.12.28).csv'
tin = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False)
tin.info()

MemoryError: Unable to allocate 168. MiB for an array with shape (22044208,) and data type int64

In [ ]:
tin.columns

Index(['사용연료', '주행거리', '차명', '차량번호', '차대번호', '차종', '차량연식', '엔진형식', '검사방법',
       '검사일자', '검사종류', '검사판정시각', '검사판정', '산소값', '이산화탄소값', '무부하매연판정1',
       '무부하매연판정2', '무부하매연판정3', '무부하매연판정4', '무부하매연판정5', '무부하매연판정6', '무부하매연허용치1',
       '무부하매연허용치2', '무부하매연허용치3', '무부하매연허용치4', '무부하매연허용치5', '무부하매연허용치6',
       '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3', '무부하매연측정치4', '무부하매연측정치5',
       '무부하매연측정치6', '차량용도', '최대출력허용치', '최대출력측정값', '검사접수번호', '검사소명', '재검사기간',
       '정기정밀', '검사 검사소코드', '검사유효기간'],
      dtype='object')

## 최근 3년 검사 데이터 추출

In [ ]:
tin['검사일자'].describe()

count   22044208.00
mean    20213557.70
std         8679.78
min     20190102.00
25%     20210427.00
50%     20211222.00
75%     20220614.00
max     20230107.00
Name: 검사일자, dtype: float64

In [ ]:
tin1 = tin[(tin['검사일자'] >= 20200101) & (tin['검사일자'] <= 20221231)]
tin1.shape

(20547872, 42)

In [ ]:
tin1['검사일자'].describe()

count   20547872.00
mean    20214858.24
std         6580.17
min     20200102.00
25%     20210611.00
50%     20220113.00
75%     20220620.00
max     20221231.00
Name: 검사일자, dtype: float64

In [ ]:
g4 = total[total['등급'] == '4'].reset_index(drop=True)
g4.shape

In [ ]:
g5.shape

In [ ]:
g4['연료'].value_counts(dropna=False)

경유             1152697
휘발유             332313
LPG(액화석유가스)      26089
NaN                 71
기타연료                39
CNG(압축천연가스)          1
알코올                  1
Name: 연료, dtype: int64

In [ ]:
g5['연료'].value_counts(dropna=False)

경유             1124188
휘발유              30040
LPG(액화석유가스)        436
NaN                 29
기타연료                25
CNG(압축천연가스)          5
알코올                  1
Name: 연료, dtype: int64

In [ ]:
g4.shape

(1511251, 49)

In [ ]:
g4 = g4.sort_values(['최초등록일자', '배출가스검사유효일자'], ascending=[False, False]).drop_duplicates('차대번호').reset_index(drop=True)
g4.shape

(1511211, 49)

In [ ]:
g5.shape

(1154762, 49)

In [ ]:
g5 = g5.sort_values(['최초등록일자', '배출가스검사유효일자'], ascending=[False, False]).drop_duplicates('차대번호').reset_index(drop=True)
g5.shape

(1154724, 49)

In [ ]:
# about 47s
g4_tin = g4.merge(tin1, on='차대번호', how='left')
g4_tin.shape

(1511211, 90)

In [ ]:
# about 43s
g5_tin = g5.merge(tin1, on='차대번호', how='left')
g5_tin.shape

(1154724, 90)

In [ ]:
g4_tin['검사방법'].unique()

array([nan, '부하검사(KD-147)', '부하검사(LUG DOWN)', '무부하검사(급가속)',
       '부하검사(ASM-Idling)', '무부하검사(TSI)', '무부하검사(정지가동)'], dtype=object)

In [ ]:
g4_tin['검사종류'].unique()

array([nan, '2부재검사(종합)', '종합검사(경과)', '종합검사', '정기검사', '정기(경과)검사',
       '재검사(2부재검사)', '재검사(1부재검사)', '1부재검사(종합)'], dtype=object)

In [ ]:
g5_tin['검사방법'].unique()

array([nan, '부하검사(KD-147)', '무부하검사(급가속)', '부하검사(LUG DOWN)', '무부하검사(TSI)',
       '부하검사(ASM-Idling)', '무부하검사(정지가동)'], dtype=object)

In [ ]:
g5_tin['검사종류'].unique()

array([nan, '종합검사', '2부재검사(종합)', '정기검사', '종합검사(경과)', '재검사(2부재검사)',
       '재검사(1부재검사)', '1부재검사(종합)', '정기(경과)검사'], dtype=object)

In [ ]:
view_cols = ['검사방법', '검사일자', '검사종류', '검사판정시각', '검사판정', 
'무부하매연판정1', '무부하매연측정치1',
'무부하매연판정2', '무부하매연측정치2',
'무부하매연판정3', '무부하매연측정치3',
'무부하매연판정4', '무부하매연측정치4',
'무부하매연판정5', '무부하매연측정치5',
'무부하매연판정6', '무부하매연측정치6',]

In [ ]:
g4_tin[g4_tin['연료'] == '알코올'].reset_index(drop=True)[view_cols].isnull().sum().reset_index()

,index,0
0,검사방법,0
1,검사일자,0
2,검사종류,0
3,검사판정시각,0
4,검사판정,0
5,무부하매연판정1,0
6,무부하매연측정치1,0
7,무부하매연판정2,0
8,무부하매연측정치2,0
9,무부하매연판정3,0


In [ ]:
g4_miss = pd.DataFrame({'컬럼명':view_cols})
for one in g4_tin['연료'].unique():
    temp = g4_tin[g4_tin['연료'] == one].reset_index(drop=True)[view_cols].isnull().sum().reset_index()
    temp.columns = ['컬럼명', f'4등급_{one}']
    g4_miss = g4_miss.merge(temp, on='컬럼명', how='left')

In [ ]:
g4_miss

,컬럼명,4등급_경유,4등급_휘발유,4등급_nan,4등급_기타연료,4등급_LPG(액화석유가스),4등급_CNG(압축천연가스),4등급_알코올
0,검사방법,44043,275254,0.00,37,21761,0,0
1,검사일자,44043,275254,0.00,37,21761,0,0
2,검사종류,44043,275254,0.00,37,21761,0,0
3,검사판정시각,52620,275825,0.00,37,21806,0,0
4,검사판정,44043,275254,0.00,37,21761,0,0
5,무부하매연판정1,52625,276357,0.00,38,21859,0,0
6,무부하매연측정치1,52620,276357,0.00,38,21859,0,0
7,무부하매연판정2,1108848,276368,0.00,38,21859,1,0
8,무부하매연측정치2,794724,276366,0.00,38,21859,1,0
9,무부하매연판정3,1109728,276368,0.00,38,21859,1,0


In [ ]:
g5_miss = pd.DataFrame({'컬럼명':view_cols})
for one in g5_tin['연료'].unique():
    temp = g5_tin[g5_tin['연료'] == one].reset_index(drop=True)[view_cols].isnull().sum().reset_index()
    temp.columns = ['컬럼명', f'5등급_{one}']
    g5_miss = g5_miss.merge(temp, on='컬럼명', how='left')

In [ ]:
g5_miss

,컬럼명,5등급_경유,5등급_휘발유,5등급_CNG(압축천연가스),5등급_LPG(액화석유가스),5등급_nan,5등급_기타연료,5등급_알코올
0,검사방법,417776,29670,3,433,0.00,25,1
1,검사일자,417776,29670,3,433,0.00,25,1
2,검사종류,417776,29670,3,433,0.00,25,1
3,검사판정시각,423727,29674,3,433,0.00,25,1
4,검사판정,417776,29670,3,433,0.00,25,1
5,무부하매연판정1,423936,29681,3,433,0.00,25,1
6,무부하매연측정치1,423927,29681,3,433,0.00,25,1
7,무부하매연판정2,1017450,29683,3,433,0.00,25,1
8,무부하매연측정치2,890184,29683,3,433,0.00,25,1
9,무부하매연판정3,1017776,29683,3,433,0.00,25,1


In [ ]:
g45_miss = g4_miss.merge(g5_miss, on='컬럼명', how='left')
g45_miss

,컬럼명,4등급_경유,4등급_휘발유,4등급_nan,4등급_기타연료,4등급_LPG(액화석유가스),4등급_CNG(압축천연가스),4등급_알코올,5등급_경유,5등급_휘발유,5등급_CNG(압축천연가스),5등급_LPG(액화석유가스),5등급_nan,5등급_기타연료,5등급_알코올
0,검사방법,44043,275254,0.00,37,21761,0,0,417776,29670,3,433,0.00,25,1
1,검사일자,44043,275254,0.00,37,21761,0,0,417776,29670,3,433,0.00,25,1
2,검사종류,44043,275254,0.00,37,21761,0,0,417776,29670,3,433,0.00,25,1
3,검사판정시각,52620,275825,0.00,37,21806,0,0,423727,29674,3,433,0.00,25,1
4,검사판정,44043,275254,0.00,37,21761,0,0,417776,29670,3,433,0.00,25,1
5,무부하매연판정1,52625,276357,0.00,38,21859,0,0,423936,29681,3,433,0.00,25,1
6,무부하매연측정치1,52620,276357,0.00,38,21859,0,0,423927,29681,3,433,0.00,25,1
7,무부하매연판정2,1108848,276368,0.00,38,21859,1,0,1017450,29683,3,433,0.00,25,1
8,무부하매연측정치2,794724,276366,0.00,38,21859,1,0,890184,29683,3,433,0.00,25,1
9,무부하매연판정3,1109728,276368,0.00,38,21859,1,0,1017776,29683,3,433,0.00,25,1


In [ ]:
today_date = datetime.today().strftime("%Y.%m.%d_%p.%H.%M.%S")
today_date

'2023.02.22_PM.15.33.36'

In [ ]:
g45_miss.to_excel(f'analysis2/4_5등급_검사_결측치_{today_date}.xlsx', index=False)

# 코드 마지막